In [3]:
pip install torchvision


/Users/pragunisanotra/.zshenv:export:2: not valid in this context: Classification/google_image_scrapping
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 4.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0
    Uninstalling torch-2.4.0:
      Successfully uninstalled torch-2.4.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torchvision import transforms, models
from torch import nn


In [2]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1000)  # Adjust the output size as needed
model = model.eval()

/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/pragunisanotra/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:21<00:00, 4.88MB/s]


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
def extract_features(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    return features.squeeze().numpy()

In [5]:
test_df = pd.read_csv('/Users/pragunisanotra/Desktop/Amazon ML Challenge/notebook/dataset/test.csv')

In [6]:
# Initialize lists to store predictions
predictions = []

In [8]:
# Check if 'image_filename' column exists
if 'image_filename' not in test_df.columns:
    print("Column 'image_filename' not found. Available columns are:", test_df.columns)
else:
    # Process each image in the test set
    for index, row in test_df.iterrows():
        image_filename = row['image_filename']
        
        if pd.isna(image_filename):
            print(f"Missing image filename at index {index}")
            continue
        
        image_path = f"path/to/test/images/{image_filename}"  # Adjust the path as needed
        
        if os.path.exists(image_path):
            # Proceed with further processing
            print(f"Processing {image_path}")
        else:
            print(f"File not found: {image_path}")


Column 'image_filename' not found. Available columns are: Index(['index', 'image_link', 'group_id', 'entity_name'], dtype='object')


In [10]:
print(f"Length of test_df.index: {len(test_df.index)}")
print(f"Length of predictions: {len(predictions)}")


Length of test_df.index: 131187
Length of predictions: 0


In [12]:
min_length = min(len(test_df.index), len(predictions))
output_df = pd.DataFrame({
    'index': test_df.index[:min_length],
    'prediction': predictions[:min_length]
})


In [13]:
if len(predictions) < len(test_df.index):
    predictions += [None] * (len(test_df.index) - len(predictions))  # Padding with None


In [14]:
# Save predictions to CSV
output_df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")

Predictions saved to predictions.csv
